In [0]:
from delta.tables import DeltaTable

In [0]:
deltaTable = DeltaTable.forPath(spark, "/Volumes/workspace/ecommerce/ecommerce_data/delta/ecommerce_events")

In [0]:
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data", header=True, inferSchema=True)

In [0]:
spark.table('events_delta').count()

In [0]:
%sql 
DESCRIBE HISTORY events_delta;

In [0]:
updates = events.limit(100).withColumn("price",events.price + 10)

In [0]:
deltaTable = DeltaTable.forName(spark, "events_delta")

deltaTable.alias("t").merge(
    updates.alias("s"),
    "t.user_session = s.user_session AND t.event_time = s.event_time"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

In [0]:
v0 = spark.read.format("delta").option("versionAsOf", 0).load("events_delta")
yesterday = spark.read.format("delta") \
    .option("timestampAsOf", "2024-01-01").load("events_delta")

In [0]:
spark.sql("OPTIMIZE events_delta ZORDER BY (event_type, user_id)")

In [0]:
spark.sql("VACUUM events_delta RETAIN 168 HOURS")

In [0]:
%sql
DESCRIBE HISTORY events_delta